# Installing Hub

In [ ]:
!pip3 install hub --quiet
import os 
os.kill(os.getpid(), 9) 

     |████████████████████████████████| 255 kB 7.7 MB/s 
     |████████████████████████████████| 152 kB 68.4 MB/s 
     |████████████████████████████████| 5.8 MB 36.6 MB/s 
     |████████████████████████████████| 3.0 MB 46.0 MB/s 
     |████████████████████████████████| 80 kB 8.4 MB/s 
     |████████████████████████████████| 105 kB 36.7 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 256 kB 77.0 MB/s 
     |████████████████████████████████| 81 kB 9.6 MB/s 
     |████████████████████████████████| 551 kB 72.4 MB/s 
     |████████████████████████████████| 1.8 MB 45.6 MB/s 
     |████████████████████████████████| 131 kB 49.7 MB/s 
     |████████████████████████████████| 75 kB 3.7 MB/s 
     |████████████████████████████████| 109 kB 52.3 MB/s 
     |████████████████████████████████| 8.5 MB 53.6 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 127 kB 58.3 MB/s 
     |███████████████████

# Download raw dataset

In [1]:
from IPython.display import clear_output
from google.colab import files

In [2]:
# Download dataset here
!wget https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/tywbtsjrjv-1.zip
!unzip tywbtsjrjv-1.zip
!unzip Plant_leaf_diseases_dataset_with_augmentation.zip  
!unzip Plant_leaf_diseases_dataset_without_augmentation.zip  
!rm -rf *.zip
clear_output()

In [28]:
import hub
from PIL import Image 
import numpy as np 
import os
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob

In [4]:
#@title Login to ActiveLoop

%env BUGGER_OFF=True
!activeloop login -u username -p password
!activeloop reporting --off

env: BUGGER_OFF=True
Successfully logged in to Activeloop.


# Creating dataset on hub

**Activeloop API** : https://docs.activeloop.ai/api-basics

In [6]:
change_classes = {
 'Peach___healthy' : 'Peach_healthy',
 'Strawberry___Leaf_scorch' : 'Strawberry_leaf_scorch',
 'Grape___Esca_(Black_Measles)' : 'Grape_black_measles',
 'Tomato___Septoria_leaf_spot' : 'Tomato_septoria_leaf_spot',
 'Grape___healthy' : 'Grape_healthy',
 'Tomato___healthy' : 'Tomato_healthy',
 'Peach___Bacterial_spot' : 'Peach_bacterial_spot',
 'Corn___Cercospora_leaf_spot Gray_leaf_spot' : 'Corn_gray_leaf_spot',
 'Soybean___healthy' : 'Soybean_healthy',
 'Corn___Common_rust' : 'Corn_common_rust',
 'Blueberry___healthy' : 'Blueberry_healthy',
 'Corn___healthy' : 'Corn_healthy',
 'Apple___healthy' : 'Apple_healthy',
 'Apple___Cedar_apple_rust' : 'Apple_cedar_apple_rust',
 'Background_without_leaves' : 'Background_without_leaves',
 'Tomato___Target_Spot' : 'Tomato_target_spot',
 'Pepper,_bell___healthy' : 'Pepper_healthy',
 'Grape___Black_rot' : 'Grape_black_rot',
 'Apple___Apple_scab' : 'Apple_scab',
 'Raspberry___healthy' : 'Raspberry_healthy',
 'Tomato___Early_blight' : 'Tomato_early_blight',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus' : 'Tomato_yellow_leaf_curl_virus',
 'Corn___Northern_Leaf_Blight' : 'Corn_northern_leaf_blight',
 'Potato___healthy' : 'Potato_healthy',
 'Tomato___Late_blight' : 'Tomato_late_blight',
 'Cherry___Powdery_mildew' : 'Cherry_powdery_mildew',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)' : 'Grape_leaf_blight',
 'Tomato___Leaf_Mold' : 'Tomato_leaf_mold',
 'Pepper,_bell___Bacterial_spot' : 'Pepper_bacterial_spot',
 'Potato___Late_blight' : 'Potato_late_blight',
 'Tomato___Tomato_mosaic_virus' : 'Tomato_mosaic_virus',
 'Potato___Early_blight' : 'Potato_early_blight',
 'Tomato___Bacterial_spot' : 'Tomato_bacterial_spot',
 'Strawberry___healthy' : 'Strawberry_healthy',
 'Cherry___healthy' : 'Cherry_healthy',
 'Squash___Powdery_mildew' : 'Squash_powdery_mildew',
 'Tomato___Spider_mites Two-spotted_spider_mite' : 'Tomato_spider_mites_two-spotted_spider_mite',
 'Orange___Haunglongbing_(Citrus_greening)' : 'Orange_haunglongbing',
 'Apple___Black_rot' : 'Apple_black_rot'
 }

In [9]:
class_names = list(change_classes.values())
folders = list(change_classes.keys())
print(f'folders -> {folders}')
print(f'classes -> {class_names}')

folders -> ['Peach___healthy', 'Strawberry___Leaf_scorch', 'Grape___Esca_(Black_Measles)', 'Tomato___Septoria_leaf_spot', 'Grape___healthy', 'Tomato___healthy', 'Peach___Bacterial_spot', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Soybean___healthy', 'Corn___Common_rust', 'Blueberry___healthy', 'Corn___healthy', 'Apple___healthy', 'Apple___Cedar_apple_rust', 'Background_without_leaves', 'Tomato___Target_Spot', 'Pepper,_bell___healthy', 'Grape___Black_rot', 'Apple___Apple_scab', 'Raspberry___healthy', 'Tomato___Early_blight', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Corn___Northern_Leaf_Blight', 'Potato___healthy', 'Tomato___Late_blight', 'Cherry___Powdery_mildew', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Tomato___Leaf_Mold', 'Pepper,_bell___Bacterial_spot', 'Potato___Late_blight', 'Tomato___Tomato_mosaic_virus', 'Potato___Early_blight', 'Tomato___Bacterial_spot', 'Strawberry___healthy', 'Cherry___healthy', 'Squash___Powdery_mildew', 'Tomato___Spider_mites Two-spotted_spider_

In [23]:
without_augmentation = '/content/Plant_leave_diseases_dataset_without_augmentation'
with_augmentation = '/content/Plant_leave_diseases_dataset_with_augmentation'

In [15]:
class_names.index('Tomato_healthy')

5

In [38]:
filename_path = 'hub://activeloop/plantvillage-without-augmentation'
# hub.delete(filename_path, large_ok=True)
ds = hub.dataset(filename_path)

with ds:
    ds.create_tensor('images', htype='image', sample_compression='jpg')
    ds.create_tensor('labels', htype='class_label', class_names = class_names)

    for folder in folders:
        path = os.path.join(without_augmentation, folder)
        label = change_classes[folder]
        label_index = class_names.index(label) 

        images = glob(os.path.join(path, '*.JPG'))
        print(f'{folder} -> {label} -> {label_index}')
        
        for image in images:
            ds.images.append(hub.read(image))
            ds.labels.append(label_index)

Your Hub dataset has been successfully created!
The dataset is private so make sure you are logged in!
This dataset can be visualized at https://app.activeloop.ai/activeloop/plantvillage-without-augmentation.
Peach___healthy -> Peach_healthy -> 0
Strawberry___Leaf_scorch -> Strawberry_leaf_scorch -> 1
Grape___Esca_(Black_Measles) -> Grape_black_measles -> 2
Tomato___Septoria_leaf_spot -> Tomato_septoria_leaf_spot -> 3
Grape___healthy -> Grape_healthy -> 4
Tomato___healthy -> Tomato_healthy -> 5
Peach___Bacterial_spot -> Peach_bacterial_spot -> 6
Corn___Cercospora_leaf_spot Gray_leaf_spot -> Corn_gray_leaf_spot -> 7
Soybean___healthy -> Soybean_healthy -> 8
Corn___Common_rust -> Corn_common_rust -> 9
Blueberry___healthy -> Blueberry_healthy -> 10
Corn___healthy -> Corn_healthy -> 11
Apple___healthy -> Apple_healthy -> 12
Apple___Cedar_apple_rust -> Apple_cedar_apple_rust -> 13
Background_without_leaves -> Background_without_leaves -> 14
Tomato___Target_Spot -> Tomato_target_spot -> 15
P

In [41]:
filename_path = 'hub://activeloop/plantvillage-with-augmentation'
# hub.delete(filename_path, large_ok=True)
ds = hub.dataset(filename_path)

with ds:
    ds.create_tensor('images', htype='image', sample_compression='jpg')
    ds.create_tensor('labels', htype='class_label', class_names = class_names)

    for folder in folders:
        path = os.path.join(with_augmentation, folder)
        label = change_classes[folder]
        label_index = class_names.index(label) 

        images = glob(os.path.join(path, '*.JPG'))
        print(f'{folder} -> {label} -> {label_index}')
        
        for image in images:
            ds.images.append(hub.read(image))
            ds.labels.append(label_index)

Your Hub dataset has been successfully created!
The dataset is private so make sure you are logged in!
This dataset can be visualized at https://app.activeloop.ai/activeloop/plantvillage-with-augmentation.
Peach___healthy -> Peach_healthy -> 0
Strawberry___Leaf_scorch -> Strawberry_leaf_scorch -> 1
Grape___Esca_(Black_Measles) -> Grape_black_measles -> 2
Tomato___Septoria_leaf_spot -> Tomato_septoria_leaf_spot -> 3
Grape___healthy -> Grape_healthy -> 4
Tomato___healthy -> Tomato_healthy -> 5
Peach___Bacterial_spot -> Peach_bacterial_spot -> 6
Corn___Cercospora_leaf_spot Gray_leaf_spot -> Corn_gray_leaf_spot -> 7
Soybean___healthy -> Soybean_healthy -> 8
Corn___Common_rust -> Corn_common_rust -> 9
Blueberry___healthy -> Blueberry_healthy -> 10
Corn___healthy -> Corn_healthy -> 11
Apple___healthy -> Apple_healthy -> 12
Apple___Cedar_apple_rust -> Apple_cedar_apple_rust -> 13
Background_without_leaves -> Background_without_leaves -> 14
Tomato___Target_Spot -> Tomato_target_spot -> 15
Pepp

# Testing dataset from Hub

In [42]:
filename_path = 'hub://activeloop/plantvillage-with-augmentation'
ds = hub.dataset(filename_path)

hub://activeloop/plantvillage-with-augmentation loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/activeloop/plantvillage-with-augmentation.


In [47]:
index = 5
image = ds.images[index].numpy()
label = ds.labels[index].data()